<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/Tools/DeepSeek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 How to Enable GPU Runtime in Jupyter Notebook

To utilize GPU acceleration for your notebook, follow these steps:

## 1. For Google Colab:
1. Click on `Runtime` in the top menu
2. Select `Change runtime type`
3. Choose `GPU` from the Hardware accelerator dropdown
4. Click `Save`

## 2. Verify GPU Availability
Run the following code to check if GPU is properly connected:

In [5]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Device Name: Tesla T4


# Load Model and Tokenizer
We'll load the model in 4-bit quantization to save memory. The model will be loaded with bitsandbytes quantization.

In [14]:
!pip uninstall transformers accelerate bitsandbytes
!pip install transformers accelerate bitsandbytes

Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.49.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.49.0
Found existing installation: accelerate 1.4.0
Uninstalling accelerate-1.4.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/bin/accelerate-merge-weights
    /usr/local/lib/python3.11/dist-packages/accelerate-1.4.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-1.4.0
Found existing installation: bitsandbytes 0.41.1
Uninstalling bitsandbytes-0.41.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.41.1.dist-info/*
    /usr/loca

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

# Define quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True  # Important for Qwen models
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,  # Important for Qwen models
    quantization_config=quantization_config,
    torch_dtype=torch.float16
)

# Move model to device manually if needed
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Verify the model is loaded correctly
print("Model loaded successfully.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/48.0k [00:00<?, ?B/s]

model-00001-of-000004.safetensors:   0%|          | 0.00/8.71G [00:00<?, ?B/s]

model-00002-of-000004.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

# Test the Model
Let's create a simple function to generate text and test it with a sample prompt.

In [ ]:
def generate_text(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with a sample prompt
prompt = "Explain quantum computing in simple terms:"
response = generate_text(prompt)
print(response)